In [67]:
GROQ_API_KEY = ""

In [68]:
from crewai import Agent, Task, Crew, Process
from groq import Groq
from langchain_groq import ChatGroq
import numpy as np
from crewai_tools import FileReadTool
import pandas as pd

In [69]:
llama3 = ChatGroq(temperature=0, model_name="llama3-70b-8192", api_key=GROQ_API_KEY)

### IMDB DATASET Reading

In [70]:
data = pd.read_csv("movies_metadata.csv")
data.head()

C:\Users\USER\AppData\Local\Temp\ipykernel_27928\2503762595.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [71]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [72]:
selected_columns = ["title", "vote_average", "genres", "original_language", "original_title", "release_date"]

In [73]:
data = data[selected_columns]
data.head()

,title,vote_average,genres,original_language,original_title,release_date
0,Toy Story,7.7,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,Toy Story,1995-10-30
1,Jumanji,6.9,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,Jumanji,1995-12-15
2,Grumpier Old Men,6.5,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",en,Grumpier Old Men,1995-12-22
3,Waiting to Exhale,6.1,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,Waiting to Exhale,1995-12-22
4,Father of the Bride Part II,5.7,"[{'id': 35, 'name': 'Comedy'}]",en,Father of the Bride Part II,1995-02-10


In [74]:
data.columns

Index(['title', 'vote_average', 'genres', 'original_language',
       'original_title', 'release_date'],
      dtype='object')

In [75]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              45460 non-null  object 
 1   vote_average       45460 non-null  float64
 2   genres             45466 non-null  object 
 3   original_language  45455 non-null  object 
 4   original_title     45466 non-null  object 
 5   release_date       45379 non-null  object 
dtypes: float64(1), object(5)
memory usage: 2.1+ MB


In [76]:
data.describe()

,vote_average
count,45460.000000
mean,5.618207
std,1.924216
min,0.000000
25%,5.000000
50%,6.000000
75%,6.800000
max,10.000000


### Agentic Workflow

In [77]:
warning_msg = "Size daha iyi sonuçlar verebilmem için Lütfen bir kaç film daha yazın..."
min_movie_amt = 3
top_similar_movie_amt = 250

In [78]:
while True:
    #prompt = "Yüzüklerin Efendisi Yüzük Kardeşliği, Hobbit Smaug'un Çorak Toprakları, Hobbit Beş Ordular Savaşı"
    prompt = input("Merhaba ben bir film tavsiye sistemiyim. Beğendiğiniz bir kaç filmi yazarsanız size beğenebileceğiniz filmler önerebilirim. : ")
    comma_amt = prompt.count(",")
    if comma_amt < min_movie_amt-1:
        print(warning_msg)
    else:
        break

In [79]:
prompt

'Jumanji, Recep İvedik, Star Wars I'

In [80]:
# Define the agent
movie_name_match_agent = Agent(
    role='Movie Name Matcher',
    goal='You should match given movie name in turkish to english',
    backstory="You are expert movie name matcher and you can easily say the official name of the given movie name.",
    llm=llama3, 
    verbose=True,  
)

movie_name_matching = Task(
    description=f"Give official name of the movie given in Turkish {prompt}",
    expected_output="Just give the official name according to given  movie name in Turkish",
    agent=movie_name_match_agent,
    #output_file="movie_name.txt"
)

In [81]:
official_name_crew = Crew(agents=[movie_name_match_agent],
               tasks=[movie_name_matching], 
               process=Process.sequential,
               verbose=1,
               max_rpm=29)
output = official_name_crew.kickoff()
output

2024-08-23 17:13:51,790 - 24684 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-23 17:13:51][DEBUG]: == Working Agent: Movie Name Matcher
 [2024-08-23 17:13:51][INFO]: == Starting Task: Give official name of the movie given in Turkish Jumanji, Recep İvedik, Star Wars I


> Entering new CrewAgentExecutor chain...
I'm ready to give my best answer.

Thought: I now can give a great answer

Final Answer: 
Here are the official English names of the given Turkish movie names:

* Jumanji: Jumanji
* Recep İvedik: Recep Ivedik
* Star Wars I: Star Wars: Episode I - The Phantom Menace

> Finished chain.
 [2024-08-23 17:13:52][DEBUG]: == [Movie Name Matcher] Task output: Here are the official English names of the given Turkish movie names:

* Jumanji: Jumanji
* Recep İvedik: Recep Ivedik
* Star Wars I: Star Wars: Episode I - The Phantom Menace




CrewOutput(raw='Here are the official English names of the given Turkish movie names:\n\n* Jumanji: Jumanji\n* Recep İvedik: Recep Ivedik\n* Star Wars I: Star Wars: Episode I - The Phantom Menace', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Give official name of the movie given in Turkish Jumanji, Recep İvedik, Star Wars I', summary='Give official name of the movie given in Turkish Jumanji,...', raw='Here are the official English names of the given Turkish movie names:\n\n* Jumanji: Jumanji\n* Recep İvedik: Recep Ivedik\n* Star Wars I: Star Wars: Episode I - The Phantom Menace', pydantic=None, json_dict=None, agent='Movie Name Matcher', output_format=<OutputFormat.RAW: 'raw'>)], token_usage=UsageMetrics(total_tokens=283, prompt_tokens=208, completion_tokens=75, successful_requests=1))

In [82]:
movie_list = output.raw
movie_list

'Here are the official English names of the given Turkish movie names:\n\n* Jumanji: Jumanji\n* Recep İvedik: Recep Ivedik\n* Star Wars I: Star Wars: Episode I - The Phantom Menace'

In [83]:
## Step 1: Define the combined agent that checks for series and finds similar movies
combined_movie_agent = Agent(
    role='Combined Movie Series and Genre Checker',
    goal=f"""Your goal is to first check if any of the given movies in {movie_list} is part of a series or trilogy. 
    If it is, prioritize those movies in the final list. After that, find {top_similar_movie_amt} similar movies based on the genre using {data}.
    Ensure that the final list gives higher priority to movies in the same series, and then fills the rest with similar genre movies.""",
    backstory="You are expert at checking movie series and finding similar genre movies.",
    llm=llama3, 
    verbose=True,  
)

combined_movie_task = Task(
    description=f"""First, check if any of the given movies in {movie_list} is part of a series or trilogy. 
    List these movies with priority. Then, search for additional movies that are similar in genre using {data} 
    until the total list reaches {top_similar_movie_amt} movies. Ensure that the search continues iteratively 
    until you have gathered a full list of {top_similar_movie_amt} movies. Expand the search scope as needed to achieve this.
    Finally: Sort this movies by IMDB Ratings. Ensure that no duplicated movie name exist.""",
    expected_output=f"""A list of {top_similar_movie_amt} similar genre movies with IMDB ratings, with movies in the same series prioritized.
    Structure of output: Movie Title, Release Year, IMDB rating are headers of the csv file.""",
    agent=combined_movie_agent,
    #output_file="expanded_similar_movies_list.txt"
    output_file = "similar_movies.csv"
)

## Step 2: Kickoff the combined agent workflow
optimized_crew = Crew(
    agents=[combined_movie_agent],
    tasks=[combined_movie_task], 
    process=Process.sequential,
    verbose=1,
    max_rpm=29
)

# Run the workflow and get the results
optimized_crew_result = optimized_crew.kickoff()
top_250_similar_movies = optimized_crew_result.raw

2024-08-23 17:14:02,519 - 24684 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-23 17:14:02][DEBUG]: == Working Agent: Combined Movie Series and Genre Checker
 [2024-08-23 17:14:02][INFO]: == Starting Task: First, check if any of the given movies in Here are the official English names of the given Turkish movie names:

* Jumanji: Jumanji
* Recep İvedik: Recep Ivedik
* Star Wars I: Star Wars: Episode I - The Phantom Menace is part of a series or trilogy. 
    List these movies with priority. Then, search for additional movies that are similar in genre using                              title  vote_average  \
0                        Toy Story           7.7   
1                          Jumanji           6.9   
2                 Grumpier Old Men           6.5   
3                Waiting to Exhale           6.1   
4      Father of the Bride Part II           5.7   
...                            ...           ...   
45461                       Subdue           4.0   
45462          Century of Birthing           9.0   
45463                     Betrayal     

In [84]:
with open('similar_movies.csv', 'r') as file:
    lines = file.readlines()

# Step 2: Remove the first line
lines = lines[2:-1]

# Step 3: Join the remaining lines back into a single string
csv_content = ''.join(lines)

# Step 4: Write this string back to a new CSV file
with open('updated_file.csv', 'w') as file:
    file.write(csv_content)

In [85]:
# Step 1: Read the CSV file into a DataFrame
df = pd.read_csv('updated_file.csv')
df.head()
sorted_df = df.sort_values(ascending=False, by="IMDB rating")

In [86]:
top_10_movies = sorted_df[:10]
top_10_movies

,Movie Title,Release Year,IMDB rating
43,The Lord of the Rings: The Return of the King,2003.0,9.2
64,Century of Birthing,2011.0,9.0
41,The Lord of the Rings: The Fellowship of the Ring,2001.0,8.8
5,Star Wars: Episode V - The Empire Strikes Back,1980.0,8.7
42,The Lord of the Rings: The Two Towers,2002.0,8.7
4,Star Wars: Episode IV - A New Hope,1977.0,8.6
55,The Incredibles,2004.0,8.5
16,Indiana Jones and the Raiders of the Lost Ark,1981.0,8.5
6,Star Wars: Episode VI - Return of the Jedi,1983.0,8.3
59,Toy Story 3,2010.0,8.3
